In [1]:
import os
import urllib
import pandas as pd
import numpy as np
from numpy.random import uniform, binomial
from sklearn.model_selection import KFold
import math

#import warnings
#warnings.filterwarnings("ignore")

#os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
# 0 = all messages are logged (default behavior)
# 1 = INFO messages are not printed
# 2 = INFO and WARNING messages are not printed
# 3 = INFO, WARNING, and ERROR messages are not printed

import time

import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp
import tensorflow_addons as tfa
from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_probability as tfp
tfd = tfp.distributions

# To remove WARNINGS from saving the models without compiling them first
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

#print(tf.__version__)
#print(tf.test.gpu_device_name())

In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print(tf.config.list_physical_devices())

#Working with CPU for now...

Num GPUs Available:  0
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


## Reading data

In [3]:
# For Default credit

input_dim_vec = [1, 2, 4, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1 ,1 ,1 ,1 ,1 ,1, 2]
width = 350
depth = 3
latent_dim = 8


# For all tabular data sets
"""
names = ['wine', 'default_credit', 'compas', 'lsat']
widths = [350, 350, 350, 350] # Bigger than VAE because the task of modelling all conditionals is more complex
depths = [3, 3, 3, 3] # We go deeper because we are using residual models
latent_dims = [6, 8, 4, 4]
under_latent_dims = [6, 8, 4, 4] # following the original paper we set dim(u) = dim(z) with d>r [r is true manifold dim]
under_latent_dims2 = [4, 6, 3, 3] # following the original paper we set dim(u) = dim(z) with d>r [r is true manifold dim]
"""
dname = 'default_credit'
print(dname)


default_credit


In [4]:
# Load the datasets in UCI
def load_UCI(dset_name, splits=10, seed=0, separate_targets=True, save_dir='data/'):
    mkdir(save_dir)

    if dset_name == 'wine':
        if not os.path.isfile(save_dir+'winequality-red.csv'):
            urllib.request.urlretrieve("https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv",
                               filename=save_dir+'winequality-red.csv')
        data = pd.read_csv(save_dir+'winequality-red.csv', header=1, delimiter=';').values
        y_idx = [-1]

    elif dset_name == 'default_credit':
        if not os.path.isfile(save_dir + 'default of credit card clients.xls'):
            urllib.request.urlretrieve(
                "https://archive.ics.uci.edu/ml/machine-learning-databases/00350/default%20of%20credit%20card%20clients.xls",
                filename=save_dir + 'default of credit card clients.xls')
        data = pd.read_excel(save_dir + 'default of credit card clients.xls', header=[0, 1], index_col=0, # delimiter="\s+"
                             ).values
        y_idx = [-1]  # OK


    else:
        raise Exception('Dataset name doesnt match any known datasets.')

    np.random.seed(seed)
    data = data[np.random.permutation(np.arange(len(data)))] #Shuffle the data
    
    kf = KFold(n_splits=splits)
    for j, (train_index, test_index) in enumerate(kf.split(data)):

        # Not sure what separate targets is
        if separate_targets:
            x_idx = list(range(data.shape[1]))
            for e in y_idx:
                x_idx.remove(x_idx[e])

            x_idx = np.array(x_idx)
            y_idx = np.array(y_idx)
            x_train, y_train = data[train_index, :], data[train_index, :]
            x_train, y_train = x_train[:, x_idx], y_train[:, y_idx]
            x_test, y_test = data[test_index, :], data[test_index, :]
            x_test, y_test = x_test[:, x_idx], y_test[:, y_idx]

            x_means, x_stds = x_train.mean(axis=0), x_train.std(axis=0)
            y_means, y_stds = y_train.mean(axis=0), y_train.std(axis=0)

            y_stds[y_stds < 1e-10] = 1
            x_stds[x_stds < 1e-10] = 1

            x_train = ((x_train - x_means) / x_stds).astype(np.float32)
            y_train = ((y_train - y_means) / y_stds).astype(np.float32)

            x_test = ((x_test - x_means) / x_stds).astype(np.float32)
            y_test = ((y_test - y_means) / y_stds).astype(np.float32)

            return x_train, x_test, x_means, x_stds, y_train, y_test, y_means, y_stds

        else:
            x_train, x_test = data[train_index, :], data[test_index, :]
            x_means, x_stds = x_train.mean(axis=0), x_train.std(axis=0)

            x_stds[x_stds < 1e-10] = 1

            x_train = ((x_train - x_means) / x_stds).astype(np.float32)
            x_test = ((x_test - x_means) / x_stds).astype(np.float32)

            return x_train, x_test, x_means, x_stds

def mkdir(paths):
    if not isinstance(paths, (list, tuple)):
        paths = [paths]
    for path in paths:
        if not os.path.isdir(path):
            os.makedirs(path, mode=0o777)

# Not sure why this is needed
def unnormalise_cat_vars(x, x_means, x_stds, input_dim_vec):
    input_dim_vec = np.array(input_dim_vec)
    unnorm_x = np.multiply(x, x_stds) + x_means

    fixed_unnorm = unnorm_x.round()
    fixed_unnorm -= fixed_unnorm.min(axis=0).reshape([1, fixed_unnorm.shape[1]])  # this sets all mins to 0
    for idx, dims in enumerate(input_dim_vec):
        if dims > 1:
            vec = fixed_unnorm[:, idx]
            vec[vec > dims - 1] = dims - 1
            fixed_unnorm[:, idx] = vec

    x[:, input_dim_vec > 1] = fixed_unnorm[:, input_dim_vec > 1]
    return x


## Recognition (Encoder)

In [5]:
"""
The arguments sent to the different keras layers are there to mimic the Torch layers in CLUE.
"""

def create_recognition_encoder(width, depth, latent_dim, input_dim_vec):
    # Tensorflow network as one big Russian doll
    nb_inputs = sum(input_dim_vec)
    inputs = keras.Input(shape=(nb_inputs,))
    #inputs = keras.Input(shape=(None,nb_inputs))
    input = layers.Dense(width, use_bias=True, \
                         kernel_initializer = tf.keras.initializers.RandomUniform(-math.sqrt(1/nb_inputs), math.sqrt(1/nb_inputs)), \
                         bias_initializer = tf.keras.initializers.RandomUniform(-math.sqrt(1/nb_inputs), math.sqrt(1/nb_inputs))) \
                         (inputs)

    for i in range(depth-1):

        x = layers.LeakyReLU(alpha=0.01)(input)
        x = layers.BatchNormalization(momentum=0.1, epsilon=1e-5)(x)
        x = layers.Dense(width, use_bias=True, \
                         kernel_initializer = tf.keras.initializers.RandomUniform(-math.sqrt(1/width), math.sqrt(1/width)), \
                         bias_initializer = tf.keras.initializers.RandomUniform(-math.sqrt(1/width), math.sqrt(1/width))) \
                         (x)

        # Skip connection 
        x = x + input

        input = x

    # Final layers
    x = layers.LeakyReLU(alpha=0.01)(x)
    x = layers.BatchNormalization(momentum=0.1, epsilon=1e-5)(x)
    outputs = layers.Dense(latent_dim*2, use_bias=True, \
                         kernel_initializer = tf.keras.initializers.RandomUniform(-math.sqrt(1/width), math.sqrt(1/width)), \
                         bias_initializer = tf.keras.initializers.RandomUniform(-math.sqrt(1/width), math.sqrt(1/width))) \
                         (x)

    recognition_encoder = keras.Model(inputs=inputs, outputs=outputs, name="recognition_encoder_model")
    return recognition_encoder
#recognition_encoder.summary()

# keras.utils.plot_model(encoder, "encoder.png")
#keras.utils.plot_model(recognition_encoder, "recognition.png", show_shapes=True)




## Prior network (Encoder)

In [6]:
"""
The arguments sent to the different keras layers are there to mimic the Torch layers in CLUE.
"""
def create_prior_encoder(width, depth, latent_dim, input_dim_vec):
    nb_inputs = sum(input_dim_vec)*2
    inputs = keras.Input(shape=(nb_inputs,))
    #inputs = keras.Input(shape=(None,nb_inputs))
    input = layers.Dense(width, use_bias=True, \
                         kernel_initializer = tf.keras.initializers.RandomUniform(-math.sqrt(1/nb_inputs), math.sqrt(1/nb_inputs)), \
                         bias_initializer = tf.keras.initializers.RandomUniform(-math.sqrt(1/nb_inputs), math.sqrt(1/nb_inputs))) \
                         (inputs)

    for i in range(depth-1):

        x = layers.LeakyReLU(alpha=0.01)(input)
        x = layers.BatchNormalization(momentum=0.1, epsilon=1e-5)(x)
        x = layers.Dense(width, use_bias=True, \
                         kernel_initializer = tf.keras.initializers.RandomUniform(-math.sqrt(1/width), math.sqrt(1/width)), \
                         bias_initializer = tf.keras.initializers.RandomUniform(-math.sqrt(1/width), math.sqrt(1/width))) \
                         (x)

        x = x + input

        input = x

    x = layers.LeakyReLU(alpha=0.01)(x)
    x = layers.BatchNormalization(momentum=0.1, epsilon=1e-5)(x)
    outputs = layers.Dense(latent_dim*2, use_bias=True, \
                         kernel_initializer = tf.keras.initializers.RandomUniform(-math.sqrt(1/width), math.sqrt(1/width)), \
                         bias_initializer = tf.keras.initializers.RandomUniform(-math.sqrt(1/width), math.sqrt(1/width))) \
                         (x)

    prior_encoder = keras.Model(inputs=inputs, outputs=outputs, name="prior_encoder_model")
    return prior_encoder
#prior_encoder.summary()

# keras.utils.plot_model(encoder, "encoder.png")
#keras.utils.plot_model(prior_encoder, "prior.png", show_shapes=True)


## Generator (Decoder)

In [7]:
"""
The arguments sent to the different keras layers are there to mimic the Torch layers in CLUE.
"""
def create_decoder(width, depth, latent_dim, input_dim_vec):
    nb_inputs = latent_dim
    inputs = keras.Input(shape=(nb_inputs,))
    input = layers.Dense(width, use_bias=True, \
                         kernel_initializer = tf.keras.initializers.RandomUniform(-math.sqrt(1/nb_inputs), math.sqrt(1/nb_inputs)), \
                         bias_initializer = tf.keras.initializers.RandomUniform(-math.sqrt(1/nb_inputs), math.sqrt(1/nb_inputs))) \
                         (inputs)

    for i in range(depth-1):

        x = layers.LeakyReLU(alpha=0.01)(input)
        x = layers.BatchNormalization(momentum=0.1, epsilon=1e-5)(x)
        x = layers.Dense(width, use_bias=True, \
                         kernel_initializer = tf.keras.initializers.RandomUniform(-math.sqrt(1/width), math.sqrt(1/width)), \
                         bias_initializer = tf.keras.initializers.RandomUniform(-math.sqrt(1/width), math.sqrt(1/width))) \
                         (x)
        x = x + input

        input = x

    x = layers.LeakyReLU(alpha=0.01)(x)
    x = layers.BatchNormalization(momentum=0.1, epsilon=1e-5)(x)
    outputs = layers.Dense(sum(input_dim_vec), use_bias=True, \
                         kernel_initializer = tf.keras.initializers.RandomUniform(-math.sqrt(1/width), math.sqrt(1/width)), \
                         bias_initializer = tf.keras.initializers.RandomUniform(-math.sqrt(1/width), math.sqrt(1/width))) \
                         (x)

    decoder = keras.Model(inputs=inputs, outputs=outputs, name="decoder_model")
    return decoder

#decoder.summary()

# keras.utils.plot_model(model, "decoder_model.png")
#keras.utils.plot_model(decoder, "generator.png", show_shapes=True)


## Masker

In [8]:
class top_masker_tensorflow:
    """
    Returned mask is sampled from component-wise independent Bernoulli
    distribution with probability of component to be unobserved p.
    Such mask induces the type of missingness which is called
    in literature "missing completely at random" (MCAR).
    If some value in batch is missed, it automatically becomes unobserved.
    """
    def __init__(self, p):
        self.p = p

    def __call__(self, batch):
        """
        Parameters:
            - batch is a numpy array with as many rows as batch_size and as many columns as features
        
        Returned:
       
            - mask is a float32 tensor
        
        The mask seems to be random
        """
        # Generate one uniform number for each row (1xrow numpy matrix)
        pp = uniform(low=0.0, high=self.p, size=batch.shape[0]) 
        pp = np.expand_dims(pp, axis=1) # Put the number in 1x1 matrices in a 1x#row matrix
        pp = np.repeat(pp, batch.shape[1], axis=1) # Repeat the number across each row
        nan_mask = tf.math.is_nan(batch) # If nan => should be unobserved i.e. boolean True
        
        # Generate Bernoulli samples (0 or 1) from pp i.e. for each sample in batch determine if a feature is
        # observed or hidden.
        bernoulli_mask_numpy = binomial(1, pp, size=None) 
        bernoulli_mask = tf.convert_to_tensor(tf.cast(bernoulli_mask_numpy, tf.bool))
        mask = tf.math.logical_or(bernoulli_mask, nan_mask) # Logical or between bernoulli and nan mask
        
        # Logical not to invert the mask (This is done in CLUE)
        # Mask is converted to a boolean tensor with floats for element wise multiplication with the batch
        # which is done in apply mask
        # (True => 0, False => 1)
        
        #TODO: The logical_not might be unnecessary as the probability of getting a true or false is equal.
        #      perhaps this can be removed later...
        return tf.cast(tf.math.logical_not(mask), dtype=tf.float32)

## Utils

In [9]:
def gauss_cat_to_flat_mask(mask, input_dim_vec):
    output = []
    for idx, dim in enumerate(input_dim_vec):
        if dim == 1:
            """
            tf.expand_dims (axis=1) takes mask[:, idx] (batch_size,) and converts it into (64,1) e.g. [1,2,3] => [[1];[2];[3]] same as torch unsqueeze(1)
            """
            output.append(tf.expand_dims(mask[:, idx], axis=1))

        elif dim > 1: 
            """
            oh_vec = mask.new_ones(mask.shape[0], dim) * mask[:, idx].unsqueeze(1) # TODO remove
            print("mask.new_ones: ", tf.ones([mask.shape[0], dim]))
            print("Right side: ", tf.expand_dims(mask[:, idx], axis=1)) 
            
            tf.expand_dims (read comment above)
            tf.ones([mask.shape[0], dim]) creates an array of batch_size x dim with ones
            oh_vec will be mask.shape[0] x dim and contain 0 or 1 on rows depending on if mask is 0 or 1.
            """
            oh_vec = tf.ones([mask.shape[0], dim]) * tf.expand_dims(mask[:, idx], axis=1) # TODO remove
            
            #print("After oh_vec: ", oh_vec)

            output.append(oh_vec)
        else:
            raise ValueError('Error, invalid dimension value')
    return tf.concat(output, axis=1)


def gauss_cat_to_flat(x, input_dim_vec):
    """
    Parameters:
        - x: numpy array
        - input_dim_vec: list e.g. [1, 2, 4, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1 ,1 ,1 ,1 ,1 ,1, 2] credit
    Returns:
        - numpy array 
        
    Example:
        
        x:
             [-0.52121574  0.          2.          1.          1.2496392   0.01383046
              0.1105278   1.8173771   0.18815508  0.2341654   1.9953084   0.2038664
              0.31341553  0.31455126  0.32473356  0.4501966   0.45570025  0.0774322
             -0.2517514  -0.1535475   0.03951775 -0.31174627 -0.12532774  0.        ]
        
        input_dim_vec:
            [1, 2, 4, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1 ,1 ,1 ,1 ,1 ,1, 2]
        
        return:
             [-0.52121574  1.          0.          0.          0.          1.
              0.          0.          1.          0.          1.2496392   0.01383046
              0.1105278   1.8173771   0.18815508  0.2341654   1.9953084   0.2038664
              0.31341553  0.31455126  0.32473356  0.4501966   0.45570025  0.0774322
             -0.2517514  -0.1535475   0.03951775 -0.31174627 -0.12532774  1.
              0.        ]
    
    """
    output = []
    for idx, dim in enumerate(input_dim_vec):
        if dim == 1:
            output.append(tf.expand_dims(x[:, idx], axis=1))
        elif dim > 1:
            oh_vec = tf.one_hot(x[:, idx], dim) # Returns one hot encoding 0 with dim 2 -> 1 0, 1 -> 0 1
            output.append(oh_vec)
        else:
            raise ValueError('Error, invalid dimension value')
    return tf.concat(output, axis=1).numpy()


## CLASS VAEAC and loss and training

In [10]:
class VAEAC_gauss_cat(tf.keras.Model):
    def __init__(self, width, depth, latent_dim, input_dim_vec, batch_size, lr, optimizer, save_model):
        super(VAEAC_gauss_cat, self).__init__()
        self.latent_dim = latent_dim
        self.input_dim_vec = input_dim_vec
        self.recognition_encoder = create_recognition_encoder(width, depth, latent_dim, input_dim_vec)
        self.prior_encoder = create_prior_encoder(width, depth, latent_dim, input_dim_vec)
        self.decoder = create_decoder(width, depth, latent_dim, input_dim_vec)
        self.sigma_mu = 1e4
        self.sigma_sigma = 1e-4
        self.vlb_scale = 1 / len(self.input_dim_vec)
        self.batch_size = batch_size
        self.optimizer = optimizer
        self.save_model = save_model
        self.lr = lr
        

    # Inspiration taken from 
    # https://github.com/joocxi/tf2-VAEAC/blob/d2b1bbc258ec77ee0975ea7eb68e63c4efcda6f0/model/vaeac.py
    def prior_regularizer(self, prior):

        mu = tf.reshape(prior.mean(), (self.batch_size, -1))
        sigma = tf.reshape(prior.scale, (self.batch_size, -1))

        mu_regularizer = -tf.reduce_sum(tf.square(mu), -1) / (2 * self.sigma_mu ** 2)
        sigma_regularizer = tf.reduce_sum((tf.math.log(sigma) - sigma), -1) * self.sigma_sigma
        return mu_regularizer + sigma_regularizer

    def apply_mask(self, x, mask):
        return x * mask
    
    def reparameterize(self, mean, logvar):
        """
        Parameters:
            - mean: tensor of size number of samples in x times half of the number of outputs in encoder
            - logvar: tensor -//-
            
        Returns:
            - The values from the encoder as a normal distribution. Multiply eps (samples from N(0,1)) 
              with std deviation tf.exp(logvar * 0.5) and add mean.
        
        """  
        eps = tf.random.normal(shape=mean.shape)
        return eps * tf.exp(logvar * .5) + mean

    def reconstruction_loss(self, x, y):
        log_prob_vec = []
        cum_dims = 0
        reshape_dim = self.batch_size
        for idx, dims in enumerate(self.input_dim_vec):
            if dims == 1:
                # Gaussian_case
                log_prob_vec.append(tf.expand_dims(-(x[:, cum_dims] - y[:, cum_dims])**2, 1))
                
                cum_dims += 1

            elif dims > 1:
                # if x.shape[1] == y.shape[1]:
                #    raise Exception('Input and target seem to be in flat format. Need integer cat targets.'

                cce = tf.keras.losses.CategoricalCrossentropy(reduction=tf.keras.losses.Reduction.NONE, from_logits = True)
                cat_cross_entropy = -cce(tf.cast(y[:, cum_dims:cum_dims + dims], dtype=tf.int64), x[:, cum_dims:cum_dims + dims])
                
                log_prob_vec.append(tf.expand_dims(cat_cross_entropy, 1))
                cum_dims += dims

            else:
                raise ValueError('Error, invalid dimension value')

        log_prob_vec = tf.reshape(log_prob_vec, [reshape_dim, len(self.input_dim_vec)])
        log_prob_vec = tf.reduce_sum(log_prob_vec, axis= -1) # Do I want this? 
                                                                 # Yes vlb in original code does this when return
        return log_prob_vec

    """
    def generate_samples_params(self, inputs, masks, sample=1):

        #Takes a model and 

        # (batch_size, width, height, channels)
        observed_inputs = self.make_observed_inputs(inputs, masks)
        # (batch_size, width, height, 2*channels)
        observed_inputs_with_masks = tf.concat([observed_inputs, masks], axis=-1)

        prior_params = self.prior_net(observed_inputs_with_masks)

        prior_distribution = tfd.Normal(
          loc=prior_params[..., :256],
          scale=tf.clip_by_value(
            tf.nn.softplus(prior_params[..., 256:]),
            1e-3,
            tf.float32.max),
          name="priors")

        samples_params = []
        for i in range(sample):
          latent = prior_distribution.sample()
          sample_params = self.generative_net(latent)
          samples_params.append(sample_params)
        return tf.stack(samples_params, axis=1)
    """
    
    # TODO Implement correctly
    """
    def generate_sample(self, sample=1):
        if(self.proposal_distribution == None):
            raise Exception('Network has no proposal distribution. Train it first')
        else:
            latent = proposal_distribution.sample()

            generative_params = self.decoder(latent)
        
            return generative_params
    """

def eval(model, x_batch, x_flat, x_masked, mask):
    """
    self.set_mode_train(train=False)

    if self.flatten:
        mask = gauss_cat_to_flat_mask(mask, self.input_dim_vec)
        x_flat = gauss_cat_to_flat(x, self.input_dim_vec)
    else:
        x_flat = x
        x = flat_to_gauss_cat(x, self.input_dim_vec)

    x_flat, x, mask = to_variable(var=(x_flat, x, mask), cuda=self.cuda)

    prior = self.model.prior_encode(x_flat, mask)

    approx_post = self.model.recognition_encode(x_flat)

    if sample:
        z_sample = approx_post.sample()
    else:
        z_sample = approx_post.loc

    rec_params = self.model.decode(z_sample)

    vlb = self.model.vlb(prior, approx_post, x, rec_params)

    if self.pred_sig:
        rec_return = normal_parse_params(rec_params, 1e-3)
    else:
        rec_return = rec_params
    return vlb.mean().item(), rec_return
    """

    #print("Validation, x_masked: ", x_masked)
    #print("Validation, x_flat: ", x_flat)
    #print("-----")
    #time.sleep(3)

    x_flat = tf.convert_to_tensor(x_flat)


    prior_params = model.prior_encoder(x_masked, training = True)
    #prior_params = model.prior_encoder(x_masked, training = False)

    proposal_params = model.recognition_encoder(x_flat, training = True)
    #proposal_params = model.recognition_encoder(x_flat, training = False)

    # Essentially CLUEs normal_parse_params
    prior_distribution = tfd.Normal(
      loc=prior_params[..., :model.latent_dim],
      scale=tf.clip_by_value(
        tf.nn.softplus(prior_params[..., model.latent_dim:]),
        1e-3,
        tf.float32.max),
      name="priors")

    proposal_distribution = tfd.Normal(
      loc=proposal_params[..., :model.latent_dim],
      scale=tf.clip_by_value(
        tf.nn.softplus(proposal_params[..., model.latent_dim:]),
        1e-3,
        tf.float32.max),
      name="proposal")

    z_sample = proposal_distribution.loc # TODO, check if this works

    #vlb = self.model.vlb(prior, approx_post, x, rec_params)
    """
    def vlb(self, prior, approx_post, x, rec_params):
        rec = self.rec_loglike(rec_params, x).view(x.shape[0], -1).sum(-1)
        prior_regularization = self.reg_cost(prior).view(x.shape[0], -1).sum(-1)
        kl = kl_divergence(approx_post, prior).view(x.shape[0], -1).sum(-1)

        return rec - kl + prior_regularization
    """

    #rec_params = model.decoder(z_sample, training = False)
    rec_params = model.decoder(z_sample, training = True)

    regularizer = model.prior_regularizer(prior_distribution)

    rec_loss = model.reconstruction_loss(rec_params, x_flat) # TODO Maybe this should be x_batch (not flattened)

    # True distribution, Estimated distribution in this order
    kl_divergence = tf.reduce_sum(
      tf.reshape(
        tfd.kl_divergence(proposal_distribution, prior_distribution),
        (model.batch_size, -1)), -1)

    vlb = tf.reduce_mean(-kl_divergence + rec_loss + regularizer) # For comparing
    return vlb, kl_divergence, rec_loss, regularizer

def compute_loss_VAEAC(model, x_batch, x_flat, x_masked, mask):

    #print("Training, x_masked: ", x_masked)
    #print("Training, x_flat: ", x_flat)
    
    #time.sleep(3)
    prior_params = model.prior_encoder(x_masked, training = True) 
    proposal_params = model.recognition_encoder(x_flat, training = True)

    # Essentially CLUEs normal_parse_params
    proposal_distribution = tfd.Normal(
      loc=proposal_params[..., :model.latent_dim],
      scale=tf.clip_by_value(
        tf.nn.softplus(proposal_params[..., model.latent_dim:]),
        1e-3,
        tf.float32.max),
      name="proposal")

    prior_distribution = tfd.Normal(
      loc=prior_params[..., :model.latent_dim],
      scale=tf.clip_by_value(
        tf.nn.softplus(prior_params[..., model.latent_dim:]),
        1e-3,
        tf.float32.max),
      name="priors")

    regularizer = model.prior_regularizer(prior_distribution)

    #print("regularizer: ", regularizer)

    latent = proposal_distribution.sample()

    generative_params = model.decoder(latent, training = True)

    rec_loss = model.reconstruction_loss(generative_params, x_flat) # TODO Maybe this should be x_batch (not flattened)
    
    #print("Prior distr: ", prior_distribution)
    #print("Proposal distr: ", proposal_distribution)
    
    # True distribution, Estimated distribution in this order
    kl_divergence = tf.reduce_sum(
      tf.reshape(
        tfd.kl_divergence(proposal_distribution, prior_distribution),
        (model.batch_size, -1)), -1)

    vlb = tf.reduce_mean(-kl_divergence + rec_loss + regularizer) # For comparing
    loss = tf.reduce_mean((kl_divergence - rec_loss - regularizer) * model.vlb_scale) 
    return loss, vlb, kl_divergence, rec_loss, regularizer

@tf.function # Converts all numpy arrays to tensors
def train_step_VAEAC(model, x_batch, x_flat, x_masked, mask):

    with tf.GradientTape() as tape:
        loss, vlb, kl_divergence, rec_loss, regularizer = compute_loss_VAEAC(model, x_batch, x_flat, x_masked, mask)
    
    # TODO train_loss(loss)???
    
    gradients = tape.gradient(loss, model.trainable_variables)
    model.optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss, vlb, kl_divergence, rec_loss, regularizer

In [11]:
def batch(iterable, n=1):
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx:min(ndx + n, l)]

def train_VAEAC(model, x_train, x_test, masker, nb_epochs, early_stop = None):
    
    vlb_train = np.zeros(nb_epochs)
    vlb_dev = np.zeros(nb_epochs)
    best_vlb = -np.inf
    best_epoch = 0
    
    test_data = []
    for x in batch(x_test, n = model.batch_size):
        test_data.append(x)
    
    for epoch in range(0, nb_epochs):
        
        
        # Shuffle the training data and sort it into batches every epoch
        train_data = []
        np.random.shuffle(x_train)
        for x in batch(x_train, n = model.batch_size):
            train_data.append(x)
        
        tic = time.time()
        ## Training
        nb_samples = 0
        for x_batch in train_data:

            model.batch_size = x_batch.shape[0] # TODO: FIX THIS: Very ugly solution now to make sure batches 
                                                # that do not have the full size
            
            mask = masker(x_batch) #tensor with floats
            
            # Flatten the batch
            x_batch_flat = gauss_cat_to_flat(x_batch, model.input_dim_vec) # numpy
            
            # Flatten the mask
            mask_flat = gauss_cat_to_flat_mask(mask, model.input_dim_vec)
            
            # Mask flattened batch
            x_batch_flat_masked = model.apply_mask(tf.convert_to_tensor(x_batch_flat), mask_flat)
            
            # Concat the mask flattened batch with the flattened mask
            x_batch_flat_masked_concat = tf.concat([x_batch_flat_masked, mask_flat], axis=1)
            
            loss, vlb, kl_divergence, rec_loss, regularizer = train_step_VAEAC(model, x_batch, x_batch_flat, x_batch_flat_masked_concat, mask_flat)

            #print("kl_div: ", kl_divergence.numpy())
            #print("reg: ", regularizer.numpy())
            #print("rec_loss: ", rec_loss.numpy())
            vlb_train[epoch] += vlb.numpy() * x_batch.shape[0]
            nb_samples += x_batch.shape[0]

        vlb_train[epoch] /= nb_samples
        toc = time.time()
        print("Epoch_" + str(epoch) + ", vlb: " + str(vlb_train[epoch]) + ", took: " + str(toc-tic))
        
        ## Validation
        nb_samples = 0
        for x_batch in test_data:
            model.batch_size = x_batch.shape[0] # TODO: FIX THIS: Very ugly solution now to make sure batches 
                                                # that do not have the full size
            mask = masker(x_batch) #tensor with floats
            
            # Flatten the batch
            x_batch_flat = gauss_cat_to_flat(x_batch, model.input_dim_vec) # numpy
            
            # Flatten the mask
            mask_flat = gauss_cat_to_flat_mask(mask, model.input_dim_vec)
            
            # Mask flattened batch
            x_batch_flat_masked = model.apply_mask(tf.convert_to_tensor(x_batch_flat), mask_flat)
            
            # Concat the mask flattened batch with the flattened mask
            x_batch_flat_masked_concat = tf.concat([x_batch_flat_masked, mask_flat], axis=1)
            
            vlb, kl_divergence, rec_loss, regularizer = eval(model, x_batch, x_batch_flat, x_batch_flat_masked_concat, mask_flat)
            #if(epoch > 0):
            #    print("kl_div: ", kl_divergence.numpy())
            #    print("reg: ", regularizer.numpy())
            #    print("rec_loss: ", rec_loss.numpy())
            vlb_dev[epoch] += vlb.numpy() * x_batch.shape[0]
            nb_samples += x_batch.shape[0]

        vlb_dev[epoch] /= nb_samples
        
        if vlb_dev[epoch] > best_vlb:
            best_vlb = vlb_dev[epoch]
            best_epoch = epoch
            if(model.save_model):
                #open text file
                text_file = open("best_epoch_VAEAC_lr_" + str(model.lr) + ".txt", "w")

                #write string to file
                text_file.write(str(epoch))

                #close file
                text_file.close()

                model.recognition_encoder.save("recog_encoder_7e-4")
                model.prior_encoder.save("prior_encoder_7e-4")
                model.decoder.save("decoder_7e-4")

        print("Validation vlb: " + str(vlb_dev[epoch]) + ", Best vlb: " + str(best_vlb) + "\n")

        if early_stop is not None and (epoch - best_epoch) > early_stop:
            print("Early stopping triggered")
            break
    
    return vlb_train, vlb_dev


        
        

## Train VAEAC

In [ ]:
batch_size = 64 # TODO get batch size to work
nb_epochs = 2000 # 2000
early_stop = 200 # TODO get early stop to work
lr = 7e-4        # Maybe this should be 1e-4, but it makes the performance terrible...

optimizer = tfa.optimizers.RectifiedAdam(lr)

# set the dimensionality of the latent space to a plane for visualization later

width = 350
depth = 3
latent_dim = 8
num_examples_to_generate = 16

masker = top_masker_tensorflow(p=1)

x_train, x_test, x_means, x_stds = \
load_UCI(dset_name=dname, splits=10, seed=42, separate_targets=False, save_dir='../data/') # np.arrays

model = VAEAC_gauss_cat(width, depth, latent_dim, input_dim_vec, batch_size, lr, optimizer, save_model = True)

x_train = unnormalise_cat_vars(x_train, x_means, x_stds, input_dim_vec) # np.array
x_test = unnormalise_cat_vars(x_test, x_means, x_stds, input_dim_vec) 

vlb_train, vlb_dev = train_VAEAC(model, x_train, x_test, masker, nb_epochs, early_stop=early_stop)

2021-12-07 10:28:36.446449: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch_0, vlb: -18.364715020073785, took: 27.10701584815979


2021-12-07 10:29:07.816439: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


Validation vlb: -10.006155448913574, Best vlb: -10.006155448913574

Epoch_1, vlb: -11.508864373101128, took: 20.368751764297485
Validation vlb: -8.113049713134766, Best vlb: -8.113049713134766

Epoch_2, vlb: -10.290587674741392, took: 16.262703895568848
Validation vlb: -7.540027280171712, Best vlb: -7.540027280171712



## Loading model

In [ ]:
# To check the total number of trainable variables 
"""
total_parameters = 0
for variable in model.trainable_variables:
    # shape is an array of tf.Dimension
    shape = variable.get_shape()
    #print(shape)
    #print(len(shape))
    variable_parameters = 1
    for dim in shape:
        #print(dim)
        variable_parameters *= dim
    #print(variable_parameters)
    total_parameters += variable_parameters
print(total_parameters)

time.sleep(10)
"""

## Generate sample parameters

## UNDER VAEAC

In [12]:
class under_VAEAC(tf.keras.Model):
    def __init__(self, base_VAE, width, depth, latent_dim, batch_size, lr, optimizer, save_model = True):
        super(under_VAEAC, self).__init__()
        
        self.base_VAEAC = base_VAE
        self.input_dim = self.base_VAEAC.latent_dim # 8 for default credit
        self.width = width
        self.depth = depth
        self.latent_dim = latent_dim # 6 for default credit
        self.lr = lr
        
        self.prior_encoder = tfd.Normal(loc=tf.zeros(latent_dim), scale=tf.ones(latent_dim))
        
        # self.input_dim is put in a list to make sum(input_dim_vec in recognition_encoder work)
        self.recognition_encoder = create_recognition_encoder(width, depth, latent_dim, [self.input_dim])
        self.decoder = create_decoder(width, depth, latent_dim, [self.input_dim])
        
        self.sigma_mu = 1e4
        self.sigma_sigma = 1e-4
        
        self.vlb_scale = 1 / self.input_dim
        self.batch_size = batch_size
        self.optimizer = optimizer
        self.save_model = save_model

    # Inspiration taken from 
    # https://github.com/joocxi/tf2-VAEAC/blob/d2b1bbc258ec77ee0975ea7eb68e63c4efcda6f0/model/vaeac.py
 
    def reparameterize(self, mean, logvar):
        """
        Parameters:
            - mean: tensor of size number of samples in x times half of the number of outputs in encoder
            - logvar: tensor -//-
            
        Returns:
            - The values from the encoder as a normal distribution. Multiply eps (samples frm N(0,1)) 
              with std deviation tf.exp(logvar * 0.5) and add mean.
        
        """  
        eps = tf.random.normal(shape=mean.shape)
        return eps * tf.exp(logvar * .5) + mean

    def reconstruction_loss(self, x, y):
        log_prob_vec = []
        reshape_dim = self.batch_size
        for idx in range(self.input_dim):
            # Gaussian_case
            log_prob_vec.append(tf.expand_dims(-(x[:, idx] - y[:, idx])**2, 1))

        log_prob_vec = tf.reshape(log_prob_vec, [reshape_dim, self.input_dim])
        log_prob_vec = tf.math.reduce_sum(log_prob_vec, axis= -1)
        
        return log_prob_vec
    
    
def compute_loss_under_VAEAC(net, x_flat):


    """
    z_sample = self.base_VAEAC.recognition_encode(x).sample()
    approx_post = self.model.encode(z_sample)
    u_sample = approx_post.rsample()
    rec_params = self.model.decode(u_sample)
    """
    
    """
    z_sample = self.base_VAEAC.recognition_encode(x).sample()
    
    Have the base_VAEAC encode the data and generate a sample from the latent representation
    """
    proposal_params_VAEAC = net.base_VAEAC.recognition_encoder(x_flat) # tensor with dim (16,)
    
    # net.input_dim = base_VAEAC latent dim (8)
    # net.latent_dim = VAE latent dim (6)
    
    # Essentially CLUEs normal_parse_params
    proposal_distribution_VAEAC = tfd.Normal(
      loc=proposal_params_VAEAC[..., :net.input_dim],
      scale=tf.clip_by_value(
        tf.nn.softplus(proposal_params_VAEAC[..., net.input_dim:]),
        1e-3,
        tf.float32.max),
      name="proposal")
    
    z_sample = proposal_distribution_VAEAC.sample() # tensor with dim (8,)
    
    """
    approx_post = self.model.encode(z_sample)
    
    Have the VAE encode the sample
    """
    
    proposal_params_VAE = net.recognition_encoder(z_sample) # tensor with dim (12,)

    proposal_distribution_VAE = tfd.Normal(
      loc=proposal_params_VAE[..., :net.latent_dim],
      scale=tf.clip_by_value(
        tf.nn.softplus(proposal_params_VAE[..., net.latent_dim:]),
        1e-3,
        tf.float32.max),
      name="priors")

    u_sample = proposal_distribution_VAE.sample() # tensor with dim (6,)
    
    
    """
    rec_params = self.model.decode(u_sample)
    
    Have the VAE decode the sample
    """
    
    rec_params = net.decoder(u_sample) # tensor with dim (8,)
    
    """
    def vlb(self, prior, approx_post, x, rec_params):
        if self.pred_sig:
            rec = self.rec_loglike(rec_params, x).view(x.shape[0], -1).sum(-1)
        else:
            rec = -self.m_rec_loglike(rec_params, x).view(x.shape[0], -1).sum(-1)
        kl = kl_divergence(approx_post, prior).view(x.shape[0], -1).sum(-1)
        return rec - kl
    
    vlb = self.model.vlb(self.prior, approx_post, z_sample, rec_params)
    loss = (- vlb * self.vlb_scale).mean()
    """
    
    #rec = -self.m_rec_loglike(rec_params, x).view(x.shape[0], -1).sum(-1)
    #kl = kl_divergence(approx_post, prior).view(x.shape[0], -1).sum(-1)
    
    # True distribution, Estimated distribution in this order
    kl_divergence = tf.reduce_sum(
      tf.reshape(
        tfd.kl_divergence(proposal_distribution_VAE, net.prior_encoder),
        (net.batch_size, -1)), -1)
    
    rec_loss = net.reconstruction_loss(rec_params, z_sample) # TODO Maybe this should be x_batch (not flattened)
    
    vlb = tf.reduce_mean(-kl_divergence + rec_loss) # For comparing
    loss = tf.reduce_mean((kl_divergence - rec_loss) * net.vlb_scale) 
    return loss, vlb, kl_divergence, rec_loss

@tf.function # Converts all numpy arrays to tensors
def train_step_under_VAEAC(model, x_flat):

    with tf.GradientTape() as tape:
        loss, vlb, kl_divergence, rec_loss = compute_loss_under_VAEAC(model, x_flat)
    
    # TODO train_loss(loss)???
    
    gradients = tape.gradient(loss, model.trainable_variables)
    model.optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss, vlb, kl_divergence, rec_loss

In [13]:
def batch(iterable, n=1):
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx:min(ndx + n, l)]

def train_VAE(model, x_train, x_test, nb_epochs, early_stop = None):
    
    vlb_train = np.zeros(nb_epochs)
    vlb_dev = np.zeros(nb_epochs)
    best_vlb = -np.inf
    best_epoch = 0
    
    train_data = []
    test_data = []
    
    for x in batch(x_train, n = model.batch_size):
        train_data.append(x)
    
    for x in batch(x_test, n = model.batch_size):
        test_data.append(x)

    for epoch in range(0, nb_epochs):
        tic = time.time()
        
        ## Training
        nb_samples = 0
        for _, x_batch in enumerate(train_data):

            model.batch_size = x_batch.shape[0] # TODO: FIX THIS: Very ugly solution now to make sure batches 
                                                # that do not have the full size
            
            # Flatten the batch
            x_batch_flat = gauss_cat_to_flat(x_batch, model.base_VAEAC.input_dim_vec) # numpy
            
            loss, vlb, kl_divergence, rec_loss = train_step_under_VAEAC(model, x_batch_flat)

            #print("kl_div: ", kl_divergence.numpy())
            #print("rec_loss: ", rec_loss.numpy())
            vlb_train[epoch] += vlb.numpy() * x_batch.shape[0]
            nb_samples += x_batch.shape[0]

        vlb_train[epoch] /= nb_samples
        toc = time.time()
        print("Epoch" + str(epoch) + ", vlb: " + str(vlb_train[epoch]) + ", took: " + str(toc-tic))
        
        ## Validation
        nb_samples = 0
        for _, x_batch in enumerate(test_data):
            model.batch_size = x_batch.shape[0] # TODO: FIX THIS: Very ugly solution now to make sure batches 
                                                # that do not have the full size
            
            # Flatten the batch
            x_batch_flat = gauss_cat_to_flat(x_batch, model.base_VAEAC.input_dim_vec) # numpy
            
            loss, vlb, kl_divergence, rec_loss = train_step_under_VAEAC(model, x_batch_flat)

            vlb_dev[epoch] += vlb.numpy() * x_batch.shape[0]
            nb_samples += x_batch.shape[0]

        vlb_dev[epoch] /= nb_samples
        
        if vlb_dev[epoch] > best_vlb:
            best_vlb = vlb_dev[epoch]
            best_epoch = epoch
            if(model.save_model):
                #open text file
                text_file = open("best_epoch_2nd_VAE_lr_" + str(model.lr) + ".txt", "w")

                #write string to file
                text_file.write(str(epoch))

                #close file
                text_file.close()

                model.recognition_encoder.save("recog_encoder_2nd_VAE")
                model.prior_encoder.save("prior_encoder_2nd_VAE")
                model.decoder.save("decoder_2nd_VAE")

        print("Validation vlb: " + str(vlb_dev[epoch]) + ", Best vlb: " + str(best_vlb) + "\n")

        if early_stop is not None and (epoch - best_epoch) > early_stop:
            print("Early stopping triggered")
            break
    return vlb_train, vlb_dev


In [14]:
batch_size = 64 # TODO get batch size to work
nb_epochs = 2000 # 2000
early_stop = 200 # TODO get early stop to work
lr = 1e-4        # Maybe this should be 1e-4, but it makes the performance terrible...

optimizer_VAEAC = tfa.optimizers.RectifiedAdam(lr)

# For Default credit
input_dim_vec = [1, 2, 4, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1 ,1 ,1 ,1 ,1 ,1, 2]
width = 350
depth = 3
latent_dim = 8
num_examples_to_generate = 16

# Create new model to load in weightsinto that can then continued to be trained
model2 = VAEAC_gauss_cat(width, depth, latent_dim, input_dim_vec, batch_size, optimizer_VAEAC, save_model = False)

model2.recognition_encoder = keras.models.load_model("recog_encoder")
model2.prior_encoder = keras.models.load_model("prior_encoder")
model2.decoder = keras.models.load_model("decoder")


# No mask is used to train the 2nd lvl VAE
#masker = top_masker_tensorflow(p=1)

x_train, x_test, x_means, x_stds = \
load_UCI(dset_name=dname, splits=10, seed=42, separate_targets=False, save_dir='../data/') # np.arrays

x_train = unnormalise_cat_vars(x_train, x_means, x_stds, input_dim_vec) # np.array
x_test = unnormalise_cat_vars(x_test, x_means, x_stds, input_dim_vec) 

base_network = model2
width = 150
depth = 2
latent_dim = 6 # under_latent_dims2 = [4, 6, 3, 3], 6 for default_credit

batch_size = 128
nb_epochs = 2000
early_stop = 200
lr = 1e-4

optimizer_under_VAEAC = tfa.optimizers.RectifiedAdam(lr)

under_VAEAC_net = under_VAEAC(base_network, width, depth, latent_dim, batch_size, lr, optimizer_under_VAEAC, save_model = False)

vlb_train, vlb_dev = train_VAE(under_VAEAC_net, x_train, x_test, nb_epochs, early_stop=early_stop)

2021-12-06 15:16:52.361979: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


ValueError: in user code:

    File "/var/folders/38/m6rs_5bd7zsf3gjy2hg8y57r0000gn/T/ipykernel_91101/1694607955.py", line 146, in train_step_under_VAEAC  *
        loss, vlb, kl_divergence, rec_loss = compute_loss_under_VAEAC(model, x_flat)
    File "/var/folders/38/m6rs_5bd7zsf3gjy2hg8y57r0000gn/T/ipykernel_91101/1694607955.py", line 131, in compute_loss_under_VAEAC  *
        kl_divergence = tf.reduce_sum(
    File "/opt/anaconda3/envs/py38/lib/python3.8/site-packages/tensorflow_probability/python/distributions/kullback_leibler.py", line 100, in kl_divergence  **
        kl_t = kl_fn(distribution_a, distribution_b, name=name)
    File "/opt/anaconda3/envs/py38/lib/python3.8/site-packages/tensorflow_probability/python/distributions/normal.py", line 273, in _kl_normal_normal
        diff_log_scale = tf.math.log(a.scale) - tf.math.log(b_scale)

    ValueError: Dimensions must be equal, but are 8 and 6 for '{{node KullbackLeibler/proposal_CONSTRUCTED_AT_top_level/KullbackLeibler_a/Norma

In [ ]:
print(under_VAEAC_net.trainable_variables)

In [31]:
print(under_VAEAC_net.decoder.summary())

Model: "decoder_model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_18 (InputLayer)          [(None, 6)]          0           []                               
                                                                                                  
 dense_63 (Dense)               (None, 150)          1050        ['input_18[0][0]']               
                                                                                                  
 leaky_re_lu_46 (LeakyReLU)     (None, 150)          0           ['dense_63[0][0]']               
                                                                                                  
 batch_normalization_46 (BatchN  (None, 150)         600         ['leaky_re_lu_46[0][0]']         
 ormalization)                                                                        

In [15]:
print(under_VAEAC_net.recognition_encoder.summary())

Model: "recognition_encoder_model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 8)]          0           []                               
                                                                                                  
 dense_12 (Dense)               (None, 150)          1350        ['input_4[0][0]']                
                                                                                                  
 leaky_re_lu_9 (LeakyReLU)      (None, 150)          0           ['dense_12[0][0]']               
                                                                                                  
 batch_normalization_9 (BatchNo  (None, 150)         600         ['leaky_re_lu_9[0][0]']          
 rmalization)                                                             

In [ ]:
### Currently Working on ###

# Need to set the models to eval mode and not training during validation data 
    # Apparently batchnorm layers do different things during training/evaluation

# Something might be wrong with how the training is done with batches...

# Have I forgotten to do a tf.reduce_sum over regularisation? Don't think so... their sum(-1) on reg_cost does nothing

# Something wrong with the validation vlb results during training of the VAEAC?

# Is something wrong with the trainable variables? Should I before training use tf.Variable to make them trainable?
# https://www.tensorflow.org/api_docs/python/tf/Variable

### TODO ###

# Generator function that is used in 2nd lvl VAE 
    # do we generate sample from the 2nd lvl VAE after 2 encodes and 1 decode? After VAE_decode we get params with 
    # dimension 8 which is the latent dimension for VAEAC? Maybe we can generates samples from that afterwards...

# Compare sizes between CLUE under and TF under in under_VAEAC

# Save the vlb_train & vlb_dev after training

# Add lr to print epoch in VAEAC training

# Remove reparametrize?

# Should I have 7e-4 or 1e-4 learning rate?

# Plot the loss graph over train and validation set

# Clean up the code

# Push to github

## Ask Ali if 
    # His network has the correct structure (The decoder was wrong for VAEAC)
    # He has thought of initlialisations
    # hyperparameters, bias, epsilon, momentum etc.

# rec_los: Should the target not be flattened but instead just x_batch?
    # Don't think so, the final values look fairly similar.

# Skip connections from prior to decoder? I don't think CLUE got this to work properly...
    # Memory layer is used in Tf2 github

# How do the batches work in the network? How can we send a 64x31 batch to encoder? It should only take 31 as input

# Not sure what the TF equivalence of affine and track_running_stats is in Torch BatchNorm1D 
    
# Why does train_step only print things inside it twice for the first batch and then never for any other batch?
    
### TO IMPLEMENT ###

# Evaluation with tensorflow

# Make VAEAC work for other datasets (COMPAS)

### DONE ###
"""

√ vlb_dev is calculated using eval not fit

√ Add shuffling of the training data 

√ Train VAEAC using 7e-4 lr and save the model (with lr in its name)

√ Verify the number of trainable parameters in CLUE vs TF 
    # Exact same amount for under_VAEAC and VAEAC

√ implement under_VAEAC vlb (MSELoss and KL-divergence)

√ rsample instead of sample? (rsample for derivatives in torch, tf does not care)

√ Save the model during training (best vlb)

√ Fix so that training with 2nd lvl VAE is done without the mask

√ 1e-4 can at times get very poor vlb at the start but then recover. Why is this?   
    # Seems to have been solved by initialising the dense weights and bias weights the same way as Torch
    # I get like -19 or -20, -22 every single first epoch now

√ Validation vlb not low enough? 
    # Not terrible but indeed not as low, seems to go towards the right values at least but it happens slowly

√ Need to initialise the weights in keras dense for the neurons and bias the same way as nn.linear

√ Need to make the layers in a skip connection sequential? nn.sequential in torch
    # The russian doll effect does exactly this!
    
√ What activation is used in dense/nn.linear?
    # None
    
√ Is keras dense and torch nn.linear the same thing?
    # How are the weights inited in each? (Different ways by default but I made them init the same way)
    # They are basically the same, input to neural network nodes (bias = True add a bias node)
    # Google images for keras dense and nn.linear and you see that it is just a normal feed forward process.
"""